In [34]:
import folium
from folium.plugins import HeatMap
import pandas as pd
import time
import requests
import json

In [35]:
def get_coords_nominatim(city):
    url = "https://nominatim.openstreetmap.org/search"
    params = {"q": f"{city}, México", "format": "json", "limit": 1}
    headers = {"User-Agent": "torneo_gobierno_irapuato_frontenis_v1.0 (contacto@irapuato.gob.mx)"}

    try:
        #print("No se esta usando cache")
        response = requests.get(url, params=params, headers=headers, timeout=15)
        response.raise_for_status()  # launch an exception if the request was unsuccesful
        data = response.json()
        return [float(data[0]["lat"]), float(data[0]["lon"])] if data else None
    except requests.exceptions.RequestException as e:
        print(f"Error en {city}: {str(e)}")
        return None

#the function returns a map with folium
def players_location(df):

    # get coords (using cache)
    df["coords"] = df["city"].apply(
        lambda x: (get_coords_nominatim(x))
    )

    # Guardar el DataFrame como un archivo JSON con codificación UTF-8
    df.to_json('players_location.json', orient='records', force_ascii=False)
    return print("Json guardado correctamente")

def get_apikey():
    payload = {
        "nickname":"astra",
        "password":"W%Auity!MM(n"
    }

    try:
        response = requests.post('https://api-x90k.onrender.com/v1/auth/login', json=payload)
        response.raise_for_status()
        data = response.json()
        data = data['data']
        return data['api_key']

    except requests.exceptions.HTTPError as err:
        print(err)
    except requests.exceptions.ConnectionError:
        print("Error de conexión")
    except requests.exceptions.Timeout:
        print("Se agotó el tiempo de la solicitud")
    except Exception as e:
        print("Error inesperado: ", str(e))

    return None

In [36]:
# Obtener la API key
api_key = get_apikey()
if api_key is None:
  print("No se pudo obtener la API key.")
  exit()

headers = {
    'X-API-KEY': api_key
}

# get the first call
try:
    response = requests.get('https://api-x90k.onrender.com/v1/players', headers=headers)

    # Verificar el código de estado de la respuesta
    if response.status_code == 200:
        # Verificar si la respuesta tiene contenido
        if response.text.strip():
            first_call = response.json()
        else:
            print("La respuesta está vacía")
    else:
        print("Error:", response.text, response.status_code)

    # get players info from all pages
    pages = first_call["pagination"]["total"]
    players = []

    for i in range(1, pages + 1):
        params = {
            "page": i
        }
        response = requests.get("https://api-x90k.onrender.com/v1/players", headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            players.extend(data["data"])
        else:
            print("Error:", response.text, response.status_code)
            break

    #count the number of players by city
    cities = {}
    for player in players:
        if player["city"] in cities:
            cities[player["city"]] += 1
        else:
            cities[player["city"]] = 1

    #make a df with two columns city and number of players
    cities_df = []
    for city, count in cities.items():
        cities_df.append({
            "city": city,
            "players": count
        })
    #make a df
    cities_df = pd.DataFrame(cities_df)
    cities_df = cities_df.sort_values(by="players", ascending=False)

except requests.exceptions.RequestException as e:
    print("Error en la solicitud:", str(e))

In [38]:
cities_df.loc[cities_df['city']=='San Luis Potosí', 'city'] = 'San Luis Potosi'
cities_df.loc[cities_df['city']=='Penjámo Gto', 'city'] = 'Pénjamo'

In [39]:
cities_df = cities_df.groupby('city').sum().reset_index()
#sort df
cities_df = cities_df.sort_values(by="players", ascending=False)

In [40]:
cities_df

,city,players
7,Irapuato,40
10,Morelia,11
12,Pénjamo,7
3,Celaya,6
11,Moroleón,5
13,Querétaro,5
5,Cortazar,4
4,Ciudad de México,3
14,San Luis Potosi,3
0,Aculco,2


In [41]:
players_location(cities_df)

Json guardado correctamente
